In [43]:
from pymongo import MongoClient
from bson.objectid import ObjectId
from bson.son import SON

In [44]:
client = MongoClient("mongodb://10.1.1.9:27017")
db = client["Neo4Food"]
orders = db["Orders"]
restaurants = db["Restaurants"]
users = db["Users"]

In [65]:
query = []

unwind = {"$unwind": "$dishes"}
addFields = {"$addFields": {"dishPrice": {"$toDouble": "$dishes.price"}}}
match = {"$match": {"restaurantId": "63d92b3cc416ac8e49aec90e", "creationDate": {"$gte": "new Date(newDate()-1000*60*60*24*30)"}}}
group = {"$group": {"_id": "$restaurantId", "name": {"$first": "$restaurant"}, "gain": {"$sum": "$total"}, "currency": {"$first":"$currency"}}}
sort = {"$sort": {"gain": -1}}
project = {"$project": {"dish": "$dishes.name", "cost": "$dishPrice", "res": "$name", "currency": "$dishes.currency"}}
limit = {"$limit": 10}


# //... {$match: {creationDate: {$gtr: new Date(new Date() - 1000*60*60*24*30)}}}
# //... {$group: {_id: "$restaurantId", name: {$first:"$restaurant"}, gain: {$sum: "$total"}, currency: {$first:"$currency"}}}]
# //... {$sort: {gain: -1}},
# //... {$limit: 10}

pipeline = [match, group, sort, limit]

print(f"db.Orders.aggregate({pipeline},{{hint: 'restaurantid1creationdate1'}}).explain()['stages']")

db.Orders.aggregate([{'$match': {'restaurantId': '63d92b3cc416ac8e49aec90e', 'creationDate': {'$gte': 'new Date(newDate()-1000*60*60*24*30)'}}}, {'$group': {'_id': '$restaurantId', 'name': {'$first': '$restaurant'}, 'gain': {'$sum': '$total'}, 'currency': {'$first': '$currency'}}}, {'$sort': {'gain': -1}}, {'$limit': 10}],{hint: 'restaurantid1creationdate1'}).explain()['stages']


In [59]:
for i in orders.list_indexes():
    print(i)

SON([('v', 2), ('key', SON([('_id', 1)])), ('name', '_id_')])
SON([('v', 2), ('key', SON([('restaurantId', 1)])), ('name', 'faster aggregations by restaurantId')])
SON([('v', 2), ('key', SON([('user', 1)])), ('name', 'faster aggregation by user name'), ('sparse', False)])
SON([('v', 2), ('key', SON([('zipcode', 1)])), ('name', 'zipcode')])
SON([('v', 2), ('key', SON([('restaurantId', 1), ('creationDate', 1)])), ('name', 'Orari')])
SON([('v', 2), ('key', SON([('creationDate', 1)])), ('name', 'creationDate')])
